In [14]:
import os
import numpy as np
import pandas as pd
import nibabel as nib
import glob
import matplotlib.pyplot as plt
from PIL import Image
import matplotlib.image as mpimg
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix,precision_score,recall_score

In [15]:
classes = ['Alzheimer', 'MCI' , 'Normal']

data = {}
labels = {}
for class_ in classes:
    dirs_ = os.listdir(class_)
    data_ = []
    for dir_ in dirs_:
        files_path =os.path.join(os.path.join(class_, dir_), '*.nii')
        for file_ in glob.glob(files_path):
            temp_file = nib.load(file_).get_data()
            if temp_file.shape[0] != 64 or temp_file.shape[1] != 64 or temp_file.shape[2] !=48:
                continue
            data_.append(temp_file)
    data[class_] = np.array(data_)

In [16]:
temp_concat = np.concatenate((data['Alzheimer'][0], data['Alzheimer'][1]), axis=3)
for i in range(2, data['Alzheimer'].shape[0]):
    temp_concat = np.concatenate((temp_concat, data['Alzheimer'][i]), axis=3)
data['Alzheimer'] = temp_concat
print data['Alzheimer'].shape

(64, 64, 48, 980)


In [17]:
# temp_concat = np.concatenate((data['Alzheimer _ 70'][0], data['Alzheimer _ 70'][1]), axis=3)
# for i in range(2, data['Alzheimer _ 70'].shape[0]):
#     temp_concat = np.concatenate((temp_concat, data['Alzheimer _ 70'][i]), axis=3)
# data['Alzheimer _ 70'] = temp_concat
# print data['Alzheimer _ 70'].shape

In [18]:
temp_concat = np.concatenate((data['MCI'][0], data['MCI'][1]), axis=3)
for i in range(2, data['MCI'].shape[0]):
    temp_concat = np.concatenate((temp_concat, data['MCI'][i]), axis=3)
data['MCI'] = temp_concat
print data['MCI'].shape

(64, 64, 48, 1540)


In [19]:
# temp_concat = np.concatenate((data['MCI _ 70'][0], data['MCI _ 70'][1]), axis=3)
# for i in range(2, data['MCI _ 70'].shape[0]):
#     temp_concat = np.concatenate((temp_concat, data['MCI _ 70'][i]), axis=3)
# data['MCI _ 70'] = temp_concat
# print data['MCI _ 70'].shape

In [20]:
temp_concat = np.concatenate((data['Normal'][0], data['Normal'][1]), axis=3)
for i in range(2, data['Normal'].shape[0]):
    temp_concat = np.concatenate((temp_concat, data['Normal'][i]), axis=3)
data['Normal'] = temp_concat
print data['Normal'].shape

(64, 64, 48, 2380)


In [21]:
# temp_concat = np.concatenate((data['Normal _ 70'][0], data['Normal _ 70'][1]), axis=3)
# for i in range(2, data['Normal _ 70'].shape[0]):
#     temp_concat = np.concatenate((temp_concat, data['Normal _ 70'][i]), axis=3)
# data['Normal _ 70'] = temp_concat
# print data['Normal _ 70'].shape

In [22]:
np.save('Normal',data['Normal'])

In [23]:
np.save('Alzheimer',data['Alzheimer'])

In [24]:
np.save('MCI',data['MCI'])

## Checkpoint


In [84]:
normal=np.load('Normal.npy')

In [85]:
mci=np.load('MCI.npy')

In [86]:
alzheimer=np.load('Alzheimer.npy')

In [87]:
alzheimer.shape

(64, 64, 48, 980)

In [88]:
mci.shape

(64, 64, 48, 1540)

In [89]:
normal.shape

(64, 64, 48, 2380)

In [90]:
combined_data=[]
for i in range(alzheimer.shape[3]):
    combined_data.append((alzheimer[:,:,:,i],0))
for i in range(normal.shape[3]):
    combined_data.append((normal[:,:,:,i],2))
for i in range(mci.shape[3]):
    combined_data.append((mci[:,:,:,i],1))
    
del(alzheimer)
del(normal)
del(mci)

In [92]:
print len(combined_data)

4900


In [93]:
# shuffling of combined_data
random.shuffle(combined_data)
print len(combined_data)

4900


In [98]:
# get input and output.
X = map(lambda x: x[0],combined_data)
print np.array(X).shape
y = map(lambda x:x[1], combined_data)
print len(y)

(4900, 64, 64, 48)
4900


## Neural Network

###  Prepare Data

In [100]:
x_Train, x_val, y_Train, y_val = train_test_split(X,y, test_size = 0.1)

In [101]:
x_train, x_test, y_train, y_test = train_test_split(x_Train,y_Train, test_size = 0.1)

In [102]:
np.array(y_val).shape

(490,)

In [103]:
short_x_train=x_train[:1000]
short_y_train=y_train[:1000]

In [104]:
short_x_val=x_val[:100]
short_y_val=y_val[:100]

In [105]:
short_x_test=x_test[:1000]
short_y_test=y_test[:1000]

In [106]:
def batch_generator(X, y, batch_size,samples_per_epoch):
    number_of_batches = (samples_per_epoch)/batch_size
    counter=0
    shuffle_index = np.arange(np.shape(y)[0])
    np.random.shuffle(shuffle_index)
    X =  X[shuffle_index,:]
    y =  y[shuffle_index]
    while 1:
        index_batch = shuffle_index[batch_size*counter:batch_size*(counter+1)]
        X_batch = X[index_batch,:]
        y_batch = y[index_batch]
        counter += 1
        yield(np.array(X_batch),y_batch)
        if (counter < number_of_batches):
            np.random.shuffle(shuffle_index)
            counter=0

In [107]:
# import the necessary packages
from keras.models import Sequential
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
 
# initialize the model
model = Sequential()

#first layer
model.add(Convolution2D(20, 5, 5, border_mode="same",
    input_shape=(64, 64, 48)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

#second layer
model.add(Convolution2D(50, 5, 5, border_mode="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

#set of FC
model.add(Flatten())
model.add(Dense(500))
model.add(Activation("relu"))

# softmax classifier
model.add(Dense(3))
model.add(Activation("softmax"))

#get model summary
model.summary()

# Compile model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Using TensorFlow backend.


AttributeError: 'module' object has no attribute 'pack'

In [ ]:
model.fit_generator(generator=batch_generator(np.asarray(short_x_train), np.asarray(short_y_train), 256,np.array(short_x_train).shape[0]),
                    nb_epoch=5, samples_per_epoch=np.array(short_x_train).shape[0],
                    validation_data=batch_generator(np.asarray(short_x_val), np.asarray(short_y_val), 256,np.array(short_x_val).shape[0]),
                    nb_val_samples=np.array(short_x_val).shape[0])

/home/siddhant/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:4: UserWarning: The semantics of the Keras 2 argument  `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Update your method calls accordingly.
/home/siddhant/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:4: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(generator=<generator..., validation_data=<generator..., steps_per_epoch=1000, epochs=5, validation_steps=100)`


Epoch 1/5
   1/1000 [..............................] - ETA: 23552s - loss: 11.7108 - acc: 0.2734